In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
books=pd.read_csv('books.csv',sep=';',error_bad_lines=False)
users=pd.read_csv('users.csv',sep=';',error_bad_lines=False)
book_rating=pd.read_csv('book_rating.csv',sep=';',error_bad_lines=False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


#### Going to rename the columns for easy use

In [4]:
books.rename(columns={'Book-Title' : 'book_title',
                      'Book-Author': 'book_author',
                      'Year-Of-Publication' : 'year',
                       'Publisher': 'publisher'},inplace=True)

#### we can see that Image-URL-S,Image-URL-M,Image-URL-L columns are of no use in recommnedation. so we can drop them..

In [5]:
books.drop(['Image-URL-S','Image-URL-M','Image-URL-L'],axis=1,inplace=True)

In [6]:
books.shape

(271360, 5)

#### Lets see the users dataset

In [7]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


#### Going to rename the columns for easy use

In [8]:
users.rename(columns={'User-ID':'user_id',
                       'Location':'location',
                        'Age':'age'},inplace=True)

In [9]:
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [10]:
users.shape

(278858, 3)

#### Lets see the book_rating dataset

In [11]:
book_rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


#### Going to rename the columns for easy use

In [12]:
book_rating.rename(columns={'User-ID' : 'user_id',
                           'Book-Rating' : 'book_rating'},inplace=True)

In [13]:
book_rating.head()

,user_id,ISBN,book_rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


#### On the basis of rating we are going to recommend the books
- Suppose 1st user reads the book and give some rating to the book
- If 2nd user reads the same type of book as of 1st so we are going to recommend the book which has been read by the user 1st and have good ratings

#### Let looks at the book_rating dataset

In [14]:
book_rating.head(2)

,user_id,ISBN,book_rating
0,276725,034545104X,0
1,276726,0155061224,5


In [15]:
book_rating.shape

(1149780, 3)

#### we can see that its a huge dataset, so we are going to take those users which have given atleast 200 ratings

In [16]:
book_rating['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
271728        1
245123        1
234886        1
259466        1
187812        1
Name: user_id, Length: 105283, dtype: int64

In [17]:
rating=book_rating['user_id'].value_counts() > 200

In [18]:
y=rating[rating].index ## this gives only true values

In [19]:
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            116122,  28634, 188951,  59727, 155916, 274808,  73681,   9856,
            268622,  44296],
           dtype='int64', length=899)

In [20]:
book_rating=book_rating[book_rating['user_id'].isin (y)]

In [21]:
book_rating

,user_id,ISBN,book_rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


#### we can see that there are only 5.2 lakh out of 11.2 lakh are users which has given minimum of 200 rating

#### In this step we are going to take only those books which have minimum of 50 ratings

In [22]:
rating_with_books=book_rating.merge(books,on='ISBN')

In [23]:
rating_with_books.head()

,user_id,ISBN,book_rating,book_title,book_author,year,publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


#### Lets count the number of rating on the basis of book_title

In [24]:
rating_count=rating_with_books.groupby('book_title')['book_rating'].count().reset_index()

In [25]:
rating_count.head()

,book_title,book_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [26]:
rating_count.rename(columns={'book_rating' : 'number_of_rating'},inplace=True)

In [27]:
rating_with_books=rating_with_books.merge(rating_count,on='book_title')

In [28]:
rating_with_books.head()

,user_id,ISBN,book_rating,book_title,book_author,year,publisher,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82


In [29]:
rating_with_books=rating_with_books[rating_with_books['number_of_rating'] > 50]

In [30]:
rating_with_books.shape

(59903, 8)

In [31]:
rating_with_books.drop_duplicates(['user_id','book_title'],inplace=True)

In [32]:
rating_with_books.head()

,user_id,ISBN,book_rating,book_title,book_author,year,publisher,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82


In [33]:
rating_with_books.shape

(57952, 8)

#### This is our cleaned dataset

#### Lets see which user has given how much rating to particular book..

In [34]:
rating_pivot=rating_with_books.pivot_table(columns='user_id',index='book_title',values='book_rating')

In [35]:
rating_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN


In [36]:
rating_pivot.fillna((0),inplace=True)

In [37]:
rating_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
book_title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [38]:
rating_pivot.shape

(703, 888)

#### we can see that we have 703 books which have atleast 50 rating count and 888 users which has given atleast 200 rating (in number)

#### we are going to use nearest neighbour algorithm to cluster the books of same rating or same type for recommendation

#### As nearest neighbour is distance based algorithm so we are going to use sparse matrix because we have many zeroes in our dataset.
To reduce the computation power we are using the concept of sparse matrix.
##### Sparse Matrix
- If most of the elements of the matrix have 0 value, then it is called a sparse matrix.
- The two major benefits of using sparse matrix instead of a simple matrix are:

1) Storage: There are lesser non-zero elements than zeros and thus lesser memory can be used to store only those elements.

2) Computing time: Computing time can be saved by logically designing a data structure traversing only non-zero elements.

In [39]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(rating_pivot)

#### Going to import the Nearest Neighbour model

In [40]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [41]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [42]:
distance,suggestion = model.kneighbors(rating_pivot.iloc[230,:].values.reshape(1,-1),n_neighbors=6)

In [43]:
distance

array([[ 0.        , 21.3541565 , 23.53720459, 24.2899156 , 24.69817807,
        24.75883681]])

In [44]:
suggestion

array([[230, 173, 277, 356, 507, 114]], dtype=int64)

In [45]:
suggestion[0][0]

230

In [46]:
rating_pivot.index[suggestion[0][0]]

'Harvest'

#### So this is the concept of getting movie name from index number

In [47]:
for i in range (len(suggestion)):
    print(rating_pivot.index[suggestion[i]])

Index(['Harvest', 'Exclusive', 'Jacob Have I Loved', 'No Safe Place',
       'The Cradle Will Fall', 'Colony'],
      dtype='object', name='book_title')


In [48]:
np.where(rating_pivot.index=='The Cradle Will Fall')[0][0]

507

In [49]:
def recommend(book):
    book_id=np.where(rating_pivot.index==book)[0][0]
    distance,suggestion = model.kneighbors(rating_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
    for i in range (len(suggestion)):
        return rating_pivot.index[suggestion[i]].to_list()

In [50]:
recommend('The Cradle Will Fall')

['The Cradle Will Fall',
 'Exclusive',
 'The Long Road Home',
 'Eyes of a Child',
 'Jacob Have I Loved',
 'No Safe Place']

#### So these all are the recommendations we get for this particular book.
#### You can check for any of the book given in the dataset.